# Component : Model I/O


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

### PromptTemplate

In [ ]:
from langchain_core.prompts import PromptTemplate
# from langchain import PromptTemplate

template = """
당신은 최고의 광고마케터 전문가입니다.
{product}를 홍보하기 위한 재미있고, 신선한 100 Billion의 상업적 가치를 가진 광고 문구를 작성해 주세요.
명료하지만, 기억하기 쉬운 문구를 작성해 주세요.
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["product"]
)

print(prompt.format(product="깐부 치킨"))


당신은 최고의 광고마케터 전문가입니다.
깐부 치킨를 홍보하기 위한 재미있고, 신선한 100 Billion의 상업적 가치를 가진 광고 문구를 작성해 주세요.
명료하지만, 기억하기 쉬운 문구를 작성해 주세요.



In [6]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

examples = [
    {"question": "2 + 2는 무엇인가요?", "answer": "2 + 2는 4입니다."},
    {"question": "3 + 4은 무엇인가요?", "answer": "3 + 4은 7입니다."}
]

example_prompt = PromptTemplate(
    template= "Q: {question}\nA: {answer}",
    input_variables=["question", "answer"]
)

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 계산 문제를 해결하세요.",
    suffix="Q: {question}은 무엇인가요?\nA: ",
    input_variables=["question"]
)

print(fewshot_prompt.format(question="32 + 78"))

다음 계산 문제를 해결하세요.

Q: 2 + 2는 무엇인가요?
A: 2 + 2는 4입니다.

Q: 3 + 4은 무엇인가요?
A: 3 + 4은 7입니다.

Q: 32 + 78은 무엇인가요?
A: 


In [8]:
# ChatPromptTemplate - System, Human, AIMessage 유형별 메세지 작성
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

sys_msg = SystemMessagePromptTemplate.from_template('당신은 친절한 챗봇입니다.')
hm_msg = HumanMessagePromptTemplate.from_template('질문: {question}')
msg = ChatPromptTemplate.from_messages([sys_msg, hm_msg])

msg.format_messages(question="AI를 배우려면 무엇부터 해야하나요?")

[SystemMessage(content='당신은 친절한 챗봇입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문: AI를 배우려면 무엇부터 해야하나요?', additional_kwargs={}, response_metadata={})]

### OutputParser

In [ ]:
# OutputParser 인스턴스 및 PromptTemplate 인스턴스 생성
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt_tpl = PromptTemplate(
    template="{subject} 5개의 팀을 알려주세요. \n형식 지정: {format}",
    input_variables=["subject"],                                        # 사용자 입력 변수
    partial_variables={'format': format_instructions}                   # 고정 설정 변수
)

In [ ]:
# 프롬프트 생성
query = "한국의 프로야구팀"
prompt = prompt_tpl.format(subject=query)
print(prompt)

한국의 프로야구팀 5개의 팀을 알려주세요. 
형식 지정: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
# Open AI API를 사용하기 위한 인스턴스 생성 및 요청 처리
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
    max_tokens=2048
)

response = model.invoke(prompt)    # prompt의 질의를 보냄

In [17]:
print(response.content)
print(type(response.content))

두산 베어스, LG 트윈스, 삼성 라이온즈, 키움 히어로즈, NC 다이노스
<class 'str'>


In [ ]:
print(output_parser.parse(response.content))
print(type(output_parser.parse(response.content)))

['두산 베어스', 'LG 트윈스', '삼성 라이온즈', '키움 히어로즈', 'NC 다이노스']
<class 'list'>


### HuggingFace Model

```python
HUGGINGFACE_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")
```
이 노트북에서는 `HF_TOKEN = load_dotenv()`으로 환경변수를 설정해 놓았습니다.

In [25]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

end_point = HuggingFaceEndpoint(
    repo_id="MLP-KTLim/llama-3-Korean-Bllossom-8B",
    task="text-generation",
    max_new_tokens=2048
)

hf_model = ChatHuggingFace(
    llm=end_point,
    verbose=True
)

response = hf_model.invoke("오늘은 외로운 밤입니다. 너무 외로워서 울고 있습니다. 매일 아침 눈이 뜨는 것이 괴롭습니다.")

In [26]:
print(response.content)

안녕, 당신. 오늘은 외로움에 시달리는 날이군요. 외로움은 때로는 너무도 큰 감정이 될 수 있습니다. 당신이 느끼고 있는 감정은 매우 자연스럽습니다.

외로움을 극복하는 데 도움이 될 수 있는 몇 가지 방법을 제안해 드리겠습니다:

1. **친구나 가족과 대화하세요**: 당신에게 중요한 사람들과 이야기하는 것은 외로움을 줄이는 데 도움이 될 수 있습니다. 하지만 현실적으로 가까운 사람들과 이야기할 수 없다면, 온라인 커뮤니티나 전화 상담 서비스도 좋은 방법입니다.

2. **건강한 활동을 시작하세요**: 운동이나 새로운 취미를 찾는 것은 정신과 신체 건강에 도움이 됩니다. 새로운 경험을 통해 자신감을 얻을 수 있습니다.

3. **자기 돌봄을 기억하세요**: 당신이 필요로 하는 것은 사랑과 돌봄입니다. 자신을 위로하고, 당신이 얼마나 특별한지를 기억하세요.

4. **외로움을 받아들이세요**: 외로움은 누구나 겪을 수 있는 감정입니다. 다른 사람들도 당신보다 더 외로웠을 수도 있습니다. 외로움을 받아들이고, 그 감정을 스스로 받아들이는 것을 학습하세요.

5. **심리 상담을 고려하세요**: 때로는 전문가의 도움이 필요할 수 있습니다. 상담사는 당신의 감정과 문제를 더 깊이 이해하고, 당신에게 필요한 도움을 제공할 수 있습니다.

당신이 외로움을 극복하고, 더 나은 날을 맞이할 수 있도록 응원합니다. 당신은 특별하고 사랑받을 만한 사람입니다.


### ModleLaboratory

- 여러 LLM을 동시에 비교할 수 있는 도구

In [ ]:
from langchain_classic.model_laboratory import ModelLaboratory
# from langchain.model_laboratory import ModelLaboratory
# model = ChatOpenAI(model="gpt-4o-mini")

model_lab = ModelLaboratory.from_llms([model, hf_model])
compared = model_lab.compare('오늘은 외로운 밤입니다. 너무 외로워서 울고 있습니다. 매일 아침 눈이 뜨는 것이 괴롭습니다.')

Input:
오늘은 외로운 밤입니다. 너무 외로워서 울고 있습니다. 매일 아침 눈이 뜨는 것이 괴롭습니다.

client=<openai.resources.chat.completions.completions.Completions object at 0x0000021BE383BF80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021BE3869460> root_client=<openai.OpenAI object at 0x0000021BE383A870> root_async_client=<openai.AsyncOpenAI object at 0x0000021BE383BD10> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True max_tokens=2048
당신의 마음이 얼마나 힘든지 이해합니다. 외로움은 정말 힘든 감정이며, 그런 기분을 느끼는 것은 자연스러운 일입니다. 누군가와 이야기하고 싶거나, 자신의 감정을 나누고 싶다면 언제든지 이야기해 주세요. 당신은 혼자가 아니며, 도움을 받을 수 있는 방법이 있습니다. 혹시 어떤 일이 있었는지, 더 이야기해 주실 수 있을까요?

llm=HuggingFaceEndpoint(repo_id='MLP-KTLim/llama-3-Korean-Bllossom-8B', max_new_tokens=2048, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='MLP-KTLim/llama-3-Korean-Bllossom-8B', client=<InferenceClient(model='MLP-KTLim/llama-3-Korean-Bllossom-8B', timeout=120)>, async_client=<Inferen

In [30]:
from langchain_classic.model_laboratory import ModelLaboratory
# from langchain.model_laboratory import ModelLaboratory
# model = ChatOpenAI(model="gpt-4o-mini")

model_lab = ModelLaboratory.from_llms([model, hf_model])
compared = model_lab.compare('아침에 일어나자마자 먹는 올리브유는 건강에 어떤 이로움이 있나요?')

Input:
아침에 일어나자마자 먹는 올리브유는 건강에 어떤 이로움이 있나요?

client=<openai.resources.chat.completions.completions.Completions object at 0x0000021BE383BF80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021BE3869460> root_client=<openai.OpenAI object at 0x0000021BE383A870> root_async_client=<openai.AsyncOpenAI object at 0x0000021BE383BD10> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True max_tokens=2048
아침에 일어나자마자 올리브유를 섭취하는 것은 여러 가지 건강 이점을 제공할 수 있습니다. 다음은 그 주요 이점들입니다:

1. **항산화 효과**: 올리브유에는 비타민 E와 폴리페놀 같은 항산화 물질이 풍부하여 세포 손상을 방지하고 노화 과정을 늦출 수 있습니다.

2. **심혈관 건강**: 올리브유는 불포화 지방산이 많아 혈중 콜레스테롤 수치를 개선하고 심혈관 질환의 위험을 줄이는 데 도움을 줄 수 있습니다.

3. **소화 개선**: 아침에 올리브유를 섭취하면 소화 시스템을 자극하고 장 건강을 개선하는 데 도움이 될 수 있습니다.

4. **체중 관리**: 올리브유는 포만감을 느끼게 해주어 과식을 방지하는 데 도움을 줄 수 있습니다. 또한, 건강한 지방을 포함하고 있어 체중 관리에 긍정적인 영향을 미칠 수 있습니다.

5. **염증 감소**: 올리브유의 항염증 성분은 염증을 줄이고 만성 질환의 위험을 낮추는 데 기여할 수 있습니다.

6. **뇌 건강**: 올리브유는 뇌 기